In [1]:
source("../Data Generator.r")
source("../Longtree.r")

Warning message:
"package 'glmertree' was built under R version 3.6.1"Loading required package: lme4
Warning message:
"package 'lme4' was built under R version 3.6.1"Loading required package: Matrix
Loading required package: partykit
Warning message:
"package 'partykit' was built under R version 3.6.1"Loading required package: grid
Loading required package: libcoin
Warning message:
"package 'libcoin' was built under R version 3.6.1"Loading required package: mvtnorm
Warning message:
"package 'WGCNA' was built under R version 3.6.1"Loading required package: dynamicTreeCut
Loading required package: fastcluster

Attaching package: 'fastcluster'

The following object is masked from 'package:stats':

    hclust


Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: 'WGCNA'

The following object is masked from 'package:stats':

    cor

Warning message:
"package 'pre' was built under R v

# The same mse problem

In [3]:
mytree = readRDS("Longtree_n100.rds")
mytree$mse

[1] 83.51495

In [4]:
mytree = readRDS("Longtree_n200.rds")
mytree$mse

[1] 83.51495

In [5]:
mytree = readRDS("Longtree_n300.rds")
mytree$mse

[1] 83.51495

# The parameter problem

I found that if I use the parameters recorded in the rds file to fit a tree, I do not get the same tree. For example for n=100

In [9]:
### training, validation and test set ###
set.seed(100)
n = 100
p = 400
imp_mod = c(1,4)
var_noise = 1
data = sim_time(n=n,p=p,imp_mod=imp_mod, var_noise=var_noise)
data$time2 = (data$time)^2

# test set (used for testing performance using optimal parameters)
set.seed(101)
n_test = 100
data_test = sim_time(n=n_test,p=p,imp_mod=imp_mod, var_noise=var_noise)
data_test$time2 = (data_test$time)^2

In [8]:
mytree = readRDS("Longtree_n100.rds")
mytree[3:length(mytree)] # see the parameters

$alpha_screen
[1] 0.4

$alpha_select
[1] 0.2

$alpha_predict
[1] 0.2

$maxdepth_factor_select
[1] 0.5

$maxdepth_factor_screen
[1] 0.2

$minsize_multiplier
[1] 5

$final_selection
[1] "V3"   "V8"   "V12"  "V54"  "V97"  "V301"

$Fuzzy
[1] TRUE

$time
   user  system elapsed 
   9.50    0.01    9.74 


In [11]:
fixed_regress = c("time","time2")
fixed_split = c("treatment")
cluster = "patient"
var_select = paste("V",1:p,sep="")
# Tuning parameters on the validation set: alpha,maxdepth,Fuzzy
alpha_screen = 0.4; alpha_select = 0.2; alpha_predict = 0.2
maxdepth_factor_select = 0.5; maxdepth_factor_screen = 0.2
minsize_multiplier = 5
Fuzzy=TRUE

time = system.time({
    mytree = Longtree(data,fixed_regress=fixed_regress,fixed_split=fixed_split,
                      var_select=var_select,cluster=cluster,Fuzzy=Fuzzy,
                      maxdepth_factor_select =  maxdepth_factor_select,
                      maxdepth_factor_screen = maxdepth_factor_screen,
                      minsize_multiplier = minsize_multiplier, 
                      alpha_screen = alpha_screen,
                      alpha_select=alpha_select,alpha_predict=alpha_predict)
})
time # running time
mean((predict(mytree,newdata=data_valid,re.form=NA)-data_valid$y)**2)
# coef(mytree)
# performance of test set
mse = mean((predict(mytree,newdata=data_test,re.form=NA)-data_test$y)**2)
mse
# plot(mytree)

Use Longtree_time


boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular


after screening within modules V1 V8 V23 V78 V93 V289 V301 


boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular


final features V1 V23 V93 V301

   user  system elapsed 
   8.61    0.12    8.98 

ERROR: Error in predict.lmertree(mytree, newdata = data_valid, re.form = NA): object 'data_valid' not found


The final features is not the same as what is recorded in rds file.